Import the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, matthews_corrcoef
import os
import time
import torch
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear, ReLU
import torch.optim as optim
import torch.nn.functional as F

import torch_geometric
from torch_geometric.nn import global_mean_pool, MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import add_self_loops
from torch.cuda.amp import GradScaler, autocast

import networkx as nx
import matplotlib.pyplot as plt
import gc
import signal
import sys

### Checkpoint and HPC Timeout Handling

Since this project was trained on a High-Performance Computing (HPC) cluster, it was necessary to handle job time limits imposed by the compute partitions:

- `gpu` partition: maximum runtime of 6 hours  
- `large` partition: maximum runtime of 24 hours

Training the GNN for the IEEE 14-bus system (and larger test systems) often exceeds these limits.  
To prevent job termination and loss of progress, this section defines two key functions:

- `save_checkpoint()` — saves the model state, optimizer state, training losses, and accuracies to a file.  
- `handle_timeout()` — automatically triggers a checkpoint save and exits gracefully just before the job time expires.

By setting an alarm signal (`signal.alarm`), a checkpoint is saved 5 minutes before the time limit, allowing training to resume later from the saved state rather than restarting from scratch.

> Note:  
> This cell is specific to the HPC environment used in this study. If you are running the notebook on a different system (e.g., local machine or cloud runtime), you have to modify or remove this section as needed.


In [ ]:
# Define the save_checkpoint function
def save_checkpoint(epoch, model, optimizer, train_losses, val_losses, train_accuracies, val_accuracies, filename='foo14N.chkp'):
  """
  I saved it as foo14N.chkp because the one in the HPC documentation was called foo.chkp,
  14 stands for 14-bus, and N stands for No_PCA (as opposed to with_PCA as stated in the paper;
  this is the only ambiguous 'encoding' I promise :D)
  """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved to {filename}")

# Define the handle_timeout function
def handle_timeout(signum, frame):
    """
    This function is called when the alarm goes off
    It saves a checkpoint and exits the program
    """
    print("Time limit reached. Saving checkpoint and exiting...")
    save_checkpoint(epoch, model, optimizer, train_losses, val_losses, train_accuracies, val_accuracies, 'foo14N.chkp')
    sys.exit(0)  # exit the program

# Set the signal handler for the alarm signal
signal.signal(signal.SIGALRM, handle_timeout)

# Set the alarm to go off 5 minutes before the job's time limit (6 hours for `gpu` partition OR 24 hours for the `large` partition)
signal.alarm(6 * 60 * 60 - 5 * 60)

### GPU Setup and Dataset Loading

This section verifies GPU availability and prepares the input data for training the GNN model.

- Dataset Loading:  
  Reads the node features, edge features, and output labels directly from Parquet files.  
  These files correspond to the IEEE 14-bus system and were generated separately using MATLAB scripts.

- Tensor Conversion and Splitting:  
  Converts the loaded data into PyTorch tensors and transfers them to the GPU memory.  
  The dataset is then split into training (60%), validation (20%), and testing (20%) sets — maintaining the same order across node, edge, and output tensors.

- Reshaping:  
  Node features are reshaped to `(samples, 14, 2)` representing 14 nodes with 2 features each,  
  while edge features are reshaped to `(samples, 20, 2)` representing 20 directed edges with 2 features each.

- Memory Management:  
  After splitting, temporary variables are deleted and GPU memory is cleared to optimize performance.

> Note:  
> If you are using a different system, dataset, or bus configuration, update the file paths and reshape dimensions accordingly.

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Read parquet files (parquet are probably the most efficient (in terms of memory size) for large numerical data)
Input_data_frame = pd.read_parquet('~/Your_Path/Node_Features_case14.parquet', engine="pyarrow")
df_numbers = pd.read_parquet('~/Your_Path/Output_Labels_Encoded_case14.parquet', engine="pyarrow")
Edge_data_frame = pd.read_parquet('~/Your_Path/Edge_Features_case14.parquet', engine="pyarrow")

# Convert dataframes to GPU tensors
Input_tensor = torch.tensor(Input_data_frame.values, dtype=torch.float16, device=device)
df_numbers_tensor = torch.tensor(df_numbers.values, dtype=torch.long, device=device)  # labels are integers
Edge_tensor = torch.tensor(Edge_data_frame.values, dtype=torch.float16, device=device)

# Split data into training, validation, and test sets (directly using tensors)
X_train, X_temp, Y_train, Y_temp = train_test_split(Input_tensor, df_numbers_tensor, train_size=0.6, shuffle=False)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, train_size=0.5, shuffle=False)

Edge_train, Edge_temp, _, _ = train_test_split(Edge_tensor, df_numbers_tensor, train_size=0.6, shuffle=False)
Edge_val, Edge_test, _, _ = train_test_split(Edge_temp, Y_temp, train_size=0.5, shuffle=False)

# Reshape (on GPU)
X_train = X_train.view(-1, 14, 2)
X_val = X_val.view(-1, 14, 2)
X_test = X_test.view(-1, 14, 2)

Edge_train = Edge_train.view(-1, 20, 2)
Edge_val = Edge_val.view(-1, 20, 2)
Edge_test = Edge_test.view(-1, 20, 2)

Y_train = Y_train.squeeze()
Y_val = Y_val.squeeze()
Y_test = Y_test.squeeze()

# Print shapes to verify
print("X_train Shape:", X_train.shape)
print("X_val Shape:", X_val.shape)
print("X_test Shape:", X_test.shape)

print("Edge_train Shape:", Edge_train.shape)
print("Edge_val Shape:", Edge_val.shape)
print("Edge_test Shape:", Edge_test.shape)

del Input_data_frame, df_numbers, Edge_data_frame
del Input_tensor, df_numbers_tensor, Edge_tensor
del X_temp, Y_temp, Edge_temp

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

### Graph Construction

The following code defines the graph connectivity of the IEEE 14-bus system.  
The `edge_index` tensor encodes the directed edges of the power network in the format expected by PyTorch Geometric, where each column represents a connection between a source node and a target node.

- The first row lists source nodes, and the second row lists target nodes.  
- The indexing follows 0-based numbering, consistent with standard graph theory and PyTorch Geometric conventions (e.g., Bus 1 → index 0).

> Note:  
> For larger test systems (IEEE 33-bus, 118-bus, or 300-bus), manually defining `edge_index` is impractical.  
> In such cases, you can write a short MATLAB script to automatically extract the bus connectivity from the system data and export the edge indices in a format compatible with this code.

In [ ]:
edge_index = torch.tensor([
    [0,0,1,1,1,2,3,3,3,4,5,5,5,6,6,8,8,9,11,12],  # Source nodes
    [1,4,2,3,4,3,4,6,8,5,10,11,12,7,8,9,13,10,12,13]   # Target nodes
], dtype=torch.long).to(device)

### Data Preparation and PyTorch Geometric Loaders

This section organizes the node, edge, and label tensors into graph objects compatible with the PyTorch Geometric framework.

- `create_data_list()` function:  
  Iterates over all samples to create a list of `Data` objects, where each object contains the node feature matrix, graph connectivity, edge feature matrix, and the output label (class).

- Dataset creation:  
  Builds separate datasets for training, validation, and testing, using the splits defined earlier.

- Data loaders:  
  The `DataLoader` utility enables mini-batch processing of multiple small graphs during training and evaluation.  
  


In [ ]:
def create_data_list(X, Y, edge_features, edge_index):
    data_list = [Data(x=X[i], edge_index=edge_index, edge_attr=edge_features[i], y=Y[i]) for i in range(len(X))]
    return data_list

# Create datasets
train_data_list = create_data_list(X_train, Y_train, Edge_train, edge_index)
val_data_list = create_data_list(X_val, Y_val, Edge_val, edge_index)
test_data_list = create_data_list(X_test, Y_test, Edge_test, edge_index)

# Loaders
train_loader = DataLoader(train_data_list, batch_size=64, shuffle=False)
val_loader = DataLoader(val_data_list, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data_list, batch_size=64, shuffle=False)

### Edge-Conditioned Convolution Layer

This section defines the Edge-Conditioned Convolution (ECC) layer, a custom message-passing neural network block that dynamically adapts node updates based on both neighboring node features and edge attributes.

- Core Idea:  
  Each message passed between connected nodes depends on the source and target node features and on the properties of the edge connecting them, according to the following equation:   
  $\mathbf{h}_i^{(k+1)} = \sigma \Bigg( \underset{j\in \mathcal{N}(i)\cup\{i\}}{\Psi} \bigg( f_\omega \Big(\mathbf{h}_i^{(k)}, \mathbf{h}_j^{(k)}, \mathbf{e}_{ij}^{(k)} \Big) \bigg) \Bigg)$  

- Implementation Details:  

  - A small MLP (`self.mlp`) combines the concatenated features `[x_i, x_j, edge_attr]` to compute the messages.  
  $\mathbf{m}_{i \gets j}^{(k)}=f_{\omega} \Big(\mathbf{h}_i^{(k)} \| \mathbf{h}_j^{(k)} \| \mathbf{e}_{ij}^{(k)} \Big)$; ($f_\omega$ in this case is the MLP)

  - Self-loops are automatically added using `add_self_loops`, and zero-valued attributes are created for them to ensure dimensional consistency.  
  $\mathbf{m}_{i \gets i}^{(k)}=f_{\omega} \Big(\mathbf{h}_i^{(k)} \| \mathbf{h}_i^{(k)} \| \mathbf{e}_{ii}^{(k)} \Big)$   

  - The layer inherits from `torch_geometric.nn.MessagePassing` with aggregation mode set to `'mean'`.  
  $\mathbf{h}_i^{(k+1)}=\sigma \Bigg(\frac{1}{|\mathcal{N}(i)|} \underset{j\in \mathcal{N}(i)\cup\{i\}}{\sum}\mathbf{m}_{i \gets j}^{(k)} \Bigg)$

  - The operations involving self-loop attributes are performed in CPU memory first, then safely moved to GPU to avoid device conflicts.

> Note:  
> The input dimensions (`in_channels_node`, `in_channels_edge`, and `out_channels`) should match your dataset configuration.  
> For other network sizes or feature definitions, these values can be easily modified.

In [ ]:
class EdgeConditionedConv(MessagePassing):
    def __init__(self, in_channels_node, in_channels_edge, out_channels):
        super(EdgeConditionedConv, self).__init__(aggr='mean')

        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(2 * in_channels_node + in_channels_edge, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, out_channels)
        )

    def forward(self, x, edge_index, edge_attr):
        # Use torch.no_grad() to avoid tracking gradients when modifying edge attributes
        with torch.no_grad():
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

            # Create self-loop attributes in CPU first, then move to GPU
            loop_attr = torch.zeros((x.size(0), edge_attr.size(1)), device='cpu', dtype=edge_attr.dtype)
            edge_attr = torch.cat([edge_attr.cpu(), loop_attr], dim=0).to(edge_attr.device, non_blocking=True)

        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr):
        return self.mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))

### Model Architecture: EC-GNN for Graph-Level Classification

This module stacks two Edge-Conditioned Convolution (ECC) layers followed by global mean pooling and two fully connected (FC) layers to produce graph-level predictions (one label per sample).

Flow
1. ECC layers (`conv1`, `conv2`)  
   - Each layer updates node embeddings using messages that depend on both neighbor node features and edge attributes.  
   - Nonlinearity: LeakyReLU (0.01) after each ECC.

2. Global mean pooling  
   - `global_mean_pool(x, batch)` aggregates node embeddings into a single graph embedding per sample.  
   $\mathbf{h}_{\mathcal{G}}=\frac{1}{|\mathcal{V}|} \sum_{i\in\mathcal{V}} \mathbf{h}_i^{(k+1)}$ you can swap `global_mean_pool` for `global_add_pool` or `global_max_pool` to test alternative aggregations.

3. MLP head (`fc1`, `fc2`)  
   - Reduces the pooled embedding and maps to `out_channels` logits (number of classes).  


Notes
- Device placement is enforced at the start of `forward` to avoid CPU/GPU mismatches.  
  
- The number of hidden layers and the number of neurons per hidden layer were chosen using grid search, and the results were used here.



In [ ]:
class ECGNN(nn.Module):
    def __init__(self, in_channels_node, in_channels_edge, hidden_channels, out_channels):
        super(ECGNN, self).__init__()

        self.conv1 = EdgeConditionedConv(in_channels_node, in_channels_edge, hidden_channels)
        self.conv2 = EdgeConditionedConv(hidden_channels, in_channels_edge, hidden_channels)

        self.fc1 = nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = nn.Linear(hidden_channels // 2, out_channels)

    def forward(self, x, edge_index, edge_attr, batch):
        # Ensure data is on the correct device before computing
        x, edge_index, edge_attr, batch = x.to(device, non_blocking=True), edge_index.to(device, non_blocking=True), edge_attr.to(device, non_blocking=True), batch.to(device, non_blocking=True)

        # Graph convolutions
        x = self.conv1(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.01)

        x = self.conv2(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.01)

        # Global mean pooling
        x = global_mean_pool(x, batch)

        # Fully connected layers
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)

        return x

### Training Setup and Mixed Precision Optimization

This section initializes the EC-GNN model, optimizer, and loss function, and defines the training loop.

- Device selection:  
  The model automatically runs on GPU if available; otherwise, it falls back to CPU.

- Model configuration:*
  The `ECGNN` is instantiated with:
  - `in_channels_node = 2` → two node features (voltage magnitude and angle OR real and reactive power injection)  
  - `in_channels_edge = 2` → two edge features (real and reactive power flow OR current magnitude and angle)  
  - `hidden_channels = 128` → hidden dimensionality for learned embeddings. The number of hidden layers and the number of neurons per hidden layer were chosen using grid search, and the results were used here.  
  - `out_channels = 35` → number of output classes (for the extended case; adjust as needed)

- Optimizer and loss:  
  Uses the Adam optimizer with a learning rate of 0.001 and `CrossEntropyLoss`.

- Mixed Precision Training (AMP):  
  - Implemented via `torch.cuda.amp.GradScaler()` and `autocast()` to reduce memory usage and accelerate training on GPUs without sacrificing accuracy.  
  - The scaled gradients are safely unscaled and updated after each batch.

- Training function (`train`)  
  Performs one full epoch over the training set:
  1. Sets the model to training mode (`model.train()`).
  2. Iterates over mini-batches from `train_loader`.
  3. Computes the loss and gradients using AMP for stability.
  4. Tracks the total loss and batch-level accuracy.
  5. Explicitly deletes batch tensors after use to free GPU memory.

> Note:  
> If you are using a smaller dataset or running locally, you can disable mixed precision by removing the `autocast()` context and `GradScaler`.  

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, Optimizer, Loss Function
model = ECGNN(in_channels_node=2, in_channels_edge=2, hidden_channels=128, out_channels=35).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Gradient Scaler for Mixed Precision Training
scaler = GradScaler()

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    for data in train_loader:
        data = data.to(device, non_blocking=True)

        optimizer.zero_grad()
        with autocast():
            output = model(data.x, data.edge_index, data.edge_attr, data.batch)
            loss = criterion(output, data.y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * data.num_graphs
        correct += (output.argmax(dim=1) == data.y).sum().item()
        total_samples += data.y.size(0)

        del data  # Free up GPU memory

    accuracy = correct / total_samples
    return total_loss / len(train_loader.dataset), accuracy

### Evaluation and Checkpoint Loading

This section defines the evaluation function and the mechanism to load training checkpoints, allowing model training to resume after HPC time limits.  

The `evaluate()` function performs validation or testing on a given dataset without updating model weights.

- The model is switched to evaluation mode (`model.eval()`), disabling dropout and batch normalization updates.

- Inference with AMP: uses `torch.cuda.amp.autocast()` to perform inference in mixed precision, maintaining consistency with training while improving speed and memory efficiency.

- Loss and Accuracy:  
  For each batch:
  - Computes predictions and loss.
  - Counts correctly classified graphs.
  - Aggregates results over all samples.  
  Returns the average loss and accuracy over the dataset.

- Memory Efficiency: each batch tensor is deleted after use, freeing GPU memory.

Checkpoint Loading and Resume  
Since GNN training can exceed the maximum allowed runtime on HPC clusters, this notebook supports resuming from a saved checkpoint.

> Note:  
> - The checkpoint file name can be changed if running multiple experiments.  
> - Ensure that the same model architecture and optimizer settings are used when resuming training.

In [ ]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for data in loader:
            data = data.to(device, non_blocking=True)

            with autocast():  # Enable Mixed Precision for inference
                output = model(data.x, data.edge_index, data.edge_attr, data.batch)
                loss = criterion(output, data.y)

            total_loss += loss.item() * data.num_graphs
            correct += (output.argmax(dim=1) == data.y).sum().item()
            total_samples += data.y.size(0)

            del data  # Free up GPU memory

    accuracy = correct / total_samples
    return total_loss / len(loader.dataset), accuracy

# Define the load_checkpoint function
def load_checkpoint(filename='foo14N.chkp'):
    if os.path.exists(filename):
        checkpoint = torch.load(filename)
        print(f"Checkpoint loaded from {filename}")
        return checkpoint
    else:
        print("No checkpoint found. Starting from scratch.")
        return None

# Load checkpoint if it exists
checkpoint = load_checkpoint('foo14N.chkp')

if checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
    train_losses = checkpoint['train_losses']
    val_losses = checkpoint['val_losses']
    train_accuracies = checkpoint['train_accuracies']
    val_accuracies = checkpoint['val_accuracies']
else:
    start_epoch = 0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

### Training & Validation Loop + Final Test

This section runs the main **epoch loop** with timing, periodic checkpointing, and a final **held-out test** evaluation.  

After training completes, runs `evaluate()` on the test set and prints Test Loss and Test Accuracy on the unseen split.

Accuracy, F1, and MCC are computed to evaluate the performance of the GNN. MCC is epecially helpful when the dataset is imbalanced.


In [ ]:
# Training Loop
num_epochs = 100
start_time = time.time()  # Start timing

for epoch in range(start_epoch, num_epochs):
    epoch_start = time.time()

    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    epoch_time = time.time() - epoch_start
    print(f'Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy:.6f}, Val Accuracy: {val_accuracy:.6f}, Time: {epoch_time:.2f} sec')

    # Save checkpoint every 5 epochs (or any interval you prefer)
    if (epoch + 1) % 5 == 0:
        save_checkpoint(epoch, model, optimizer, train_losses, val_losses, train_accuracies, val_accuracies, 'foo14N.chkp')

total_time = time.time() - start_time
print(f"\nTotal training time for {num_epochs} epochs: {total_time:.2f} sec")


# Testing Loop
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}")

### Final Evaluation on Test Data

After training and validation are complete, this section performs the final evaluation of the trained EC-GNN model on the unseen test set.  
Although `evaluate()` was used during training to track progress via loss and accuracy, `evaluate_with_metrics()` provides a comprehensive, publication-ready performance summary.

It is used only once after training on the test set to quantify the model’s generalization ability. It computes additional metrics that are more informative than accuracy alone, particularly for imbalanced classes.

> Note:  
> - This evaluation is separate from the validation steps inside the training loop.  
> - It provides the final quantitative performance of the trained model on completely unseen data.

In [ ]:
def evaluate_with_metrics(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in loader:
            data = data.to(device, non_blocking=True)

            with autocast():  # Enable Mixed Precision for inference
                output = model(data.x, data.edge_index, data.edge_attr, data.batch)
                loss = criterion(output, data.y)

            total_loss += loss.item() * data.num_graphs
            preds = output.argmax(dim=1)
            correct += (preds == data.y).sum().item()
            total_samples += data.y.size(0)

            # Collect predictions and labels for F1 and MCC
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())

            del data  # Free up GPU memory

    accuracy = correct / total_samples
    avg_loss = total_loss / len(loader.dataset)

    # Calculate F1 score and MCC
    f1 = f1_score(all_labels, all_preds, average='weighted')  # Use 'weighted' for multi-class
    mcc = matthews_corrcoef(all_labels, all_preds)

    return avg_loss, accuracy, f1, mcc

# Testing Loop with F1 and MCC
test_loss, test_accuracy, test_f1, test_mcc = evaluate_with_metrics(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}, Test F1 Score: {test_f1:.6f}, Test MCC: {test_mcc:.6f}")

### 💾 Saving the Trained Model

Once training and final evaluation are complete, the trained EC-GNN model is saved for future use.  
This allows you to reload the learned weights without retraining the model from scratch.

- What is saved:  
  Only the model parameters (`state_dict`) are stored, not the optimizer or training history.  
  This compact format is ideal for inference or transfer learning on similar datasets.

> Tip:  
> To reload the model later:
> ```python
> model = ECGNN(in_channels_node=2, in_channels_edge=2, hidden_channels=128, out_channels=35)
> model.load_state_dict(torch.load('~/Your_Path/Fourteen_Bus_No_PCA.pth'))
> model.eval()
> ```


In [ ]:
# Save the trained model
model_save_path = os.path.expanduser('~/Your_Path/Fourteen_Bus_No_PCA.pth')
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")